In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0023293495178222656
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 41.42832090207411
200000 83.28600800556966
300000 94.79198914514464
400000 95.33272487639148
500000 95.12038146272857
600000 97.60888879948303
700000 94.78932003107744
800000 93.18308588188297
900000 93.15035324523885
1000000 94.55032364428764
1100000 92.72508132576776
1200000 26.060340905753257
1300000 93.80426759465043
1400000 93.44167419624392
1500000 92.80002580619883
1600000 92.90231355294928
1700000 94.18908014084239
1800000 98.25160221497138
1900000 94.1195386020447
2000000 93.99774836515961
2100000 93.6639407245869
2200000 95.1304205841594
2300000 73.99516375498195
2400000 46.067949382656494
2500000 92.60471116350158
2600000 91.68172474719809
2700000 92.17910460525172
2800000 91.28728598368741
2900000 94.88319314236786
3000000 92.28899442752514

29800000 74.64575825405491
29900000 102.32543934317673
30000000 100.9023296382644
30100000 98.70872349274009
30200000 42.26689877056635
30300000 100.4978854840922
30400000 102.19143631058445
30500000 101.21254529327392
30600000 96.4387187938831
30700000 99.7916186481189
30800000 97.294346663533
30900000 95.0027996624427
31000000 91.36829204539504
31100000 26.08395826683281
31200000 97.61874185409684
31300000 95.03324723419506
31400000 96.43749149329074
31500000 99.79686558313927
31600000 97.86419794574051
31700000 99.46627318274493
31800000 101.2119564953834
31900000 100.86940882888062
32000000 56.39546492364258
32100000 88.70260174135989
32200000 105.96866648708045
32300000 97.43236338241702
32400000 101.12787070559165
32500000 33.15329969657192
32600000 100.69488693030462
32700000 99.59922101223033
32800000 95.8831597003775
32900000 98.2705103643973
33000000 96.7586346615412
33100000 93.55876879536264
33200000 92.15817431213252
33300000 88.36609409768309
33400000 26.386023102139394
3

60200000 96.68871982437845
60300000 36.497458264794766
60400000 93.52394517918846
60500000 91.1344834594235
60600000 90.37584720862579
60700000 86.87602747748407
60800000 85.18403606062371
60900000 85.38527570969507
61000000 23.401240907205384
61100000 90.66754244857043
61200000 85.11533145061956
61300000 86.29016636360913
61400000 83.55851834571637
61500000 89.3543245556892
61600000 91.64194691708296
61700000 36.31563392751416
61800000 99.88625505096908
61900000 96.82429595795
62000000 94.20702825497149
62100000 98.3364116065808
62200000 97.07697305856857
62300000 97.29557577200347
62400000 99.75049792152369
62500000 95.77902369207378
62600000 50.46453251994526
62700000 82.03457220553886
62800000 93.33559324018385
62900000 89.67311360381281
63000000 83.66831214984627
63100000 87.44556290278554
63200000 88.30717233685768
63300000 23.101752079500496
63400000 92.93947362101053
63500000 88.85015964773841
63600000 84.27808122361108
63700000 88.55009608689018
63800000 90.36030379499192
6390

90700000 80.79114876847298
90800000 79.55940998898402
90900000 19.313798312637996
91000000 80.4895623229707
91100000 83.71551168400958
91200000 77.74902005732159
91300000 73.51702264133768
91400000 42.47511385688601
91500000 76.7817208134245
91600000 75.87273072270249
91700000 79.09388840500344
91800000 85.62358380875848
91900000 82.13103844961606
92000000 87.50227854015304
92100000 85.67192339000007
92200000 85.86732372951698
92300000 85.04425335243924
92400000 82.47343675222088
92500000 82.79886854261024
92600000 80.94519194334912
92700000 55.03137484392945
92800000 54.5640289963808
92900000 74.64774940140958
93000000 76.1642024422439
93100000 78.96519536057696
93200000 21.105112861181787
93300000 81.45295871138946
93400000 77.78380208975673
93500000 75.26913708844829
93600000 58.04776033129867
93700000 54.758681046367606
93800000 76.80432219154788
93900000 80.3930862628406
94000000 78.45946639842488
94100000 79.73481180883618
94200000 79.35038720646797
94300000 86.184506382407
94400

120300000 52.815133092355126
120400000 48.62342414326316
120500000 57.56041505099237
120600000 45.986164145101036
120700000 67.37270980440024
120800000 16.973512073332913
120900000 68.85512843271562
121000000 50.68358917553469
121100000 55.74723696533352
121200000 57.24510750681181
121300000 61.472489340774736
121400000 40.19988267497157
121500000 38.908041593755854
121600000 40.74611485790247
121700000 56.10047680424868
121800000 57.37270777652624
121900000 61.984316439963536
122000000 85.24298359826531
122100000 55.55388279972515
122200000 56.856949228141346
122300000 59.967124651555096
122400000 62.09831168046981
122500000 58.88525443937574
122600000 55.72758952020668
122700000 61.01666707301095
122800000 63.62336958378552
122900000 18.83430330850268
123000000 67.20139263473497
123100000 10.067140960911336
123200000 62.23664587098524
123300000 46.458385994176275
123400000 66.94431069727162
123500000 67.99154525581876
123600000 53.09932499650079
123700000 43.74437655150617
123800000 

...

1191100000 3.5372650536347896
1191200000 3.0308764322357677
1191300000 3.3883170958289206
1191400000 3.2156931364056898
1191500000 3.3356606726184586
1191600000 4.213961165551642
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 3.007146281018817
1191800000 1.6513050765701434
1191900000 1.2876310243557691
1192000000 1.702458917389118
1192100000 1.7689838849216866
1192200000 1.285923027167817
1192300000 1.6517991932638156
1192400000 2.2024985026485293
1192500000 1.9515061356159051
1192600000 1.33474167058847
1192700000 1.8875022035254372
1192800000 1.6609114452119171
1192900000 1.2102832333697775
1193000000 1.8247388715413577
1193100000 1.7764562570592863
1193200000 1.4680716062178445
best so far: 0
type: [1, 4, 1, 11, 3] 132
cases of this type: 431244
1193300000 2.093153039534486
1193400000 1.90748636954538
1193500000 1.9090936585102114
1193600000 1.7543706323291228
best so far: 0
type: [1, 4, 1, 33, 1] 132
cases of this type: 143748
1193700000 1.10380979

1204900000 2.341206002010468
1205000000 2.582294333849692
1205100000 3.339306538058013
1205200000 2.372125257191694
1205300000 2.750832547019157
1205400000 3.1701606862517293
1205500000 2.228341615927958
1205600000 2.8338705347689888
1205700000 3.091892878977457
1205800000 1.9605486010490887
1205900000 3.227902494075364
1206000000 2.97776472358047
1206100000 1.844338817435556
1206200000 3.309202561490781
1206300000 3.2819192116566898
1206400000 1.8708565220557682
1206500000 3.199071147016197
1206600000 2.682640624502381
1206700000 2.1096661138067283
1206800000 3.359538577578955
1206900000 2.9343301638925077
1207000000 2.6256518592776823
1207100000 2.189410154303117
1207200000 0.6017927743548751
1207300000 0.5328696777323054
1207400000 0.6920917135720022
1207500000 0.5459451451935171
1207600000 0.5154990419259071
1207700000 0.5506259418993328
1207800000 0.5202249458983541
1207900000 0.5178670038990544
1208000000 0.5827060548882054
1208100000 0.6201032507818938
1208200000 0.5934477154324

1232100000 0.9676876607118348
1232200000 0.902196385944936
1232300000 0.7050727915688819
1232400000 0.9862102234119475
1232500000 0.7363256198251644
1232600000 0.7638207237191498
1232700000 0.9002287760642965
1232800000 0.6652666500950521
1232900000 0.7912343262195091
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 0.9490142632090152
1233100000 0.5614417679233584
1233200000 0.8935952205319505
1233300000 0.9467645509940386
1233400000 0.5705046738872925
1233500000 0.8796057289460467
1233600000 0.9062645695756077
1233700000 0.5006651195249094
1233800000 0.8267992224811647
1233900000 0.8543236390871306
1234000000 0.5447297630547319
1234100000 0.8736749920524193
1234200000 0.7022055033298433
1234300000 0.6281702693531944
1234400000 0.9167957580559254
1234500000 0.7527831174968381
1234600000 0.6530167714174423
1234700000 0.8398163150462905
1234800000 0.6362153808104694
1234900000 0.6982159849884974
1235000000 0.8734385296988685
1235100000 0.6038261541164517
1

1253400000 0.24437470433368286
1253500000 0.21129939062749015
1253600000 0.26745015621969104
1253700000 0.22288019195915262
1253800000 0.24751936678283085
1253900000 0.2402553098231223
1254000000 0.21099744890874628
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.2711245259762009
1254200000 0.22311108544869224
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.15229203293032445
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.2318125512117379
1254500000 0.12692004886759983
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases of this type: 23958
best so far: 0
type: [2, 2, 11, 1, 3] 132
cases of this type: 19602
1254600000 0.25828460758700966
best so far: 0
type: [2, 2, 11, 3, 1] 132
cases of this type: 6534
best so far: 0
type: [2, 2, 33, 1, 1] 132
cases of this type: 2178
best so far: 0
type: [2, 3, 1, 1, 22] 132
cases of this type: 702768
1254700000 0.3265940014538699
1254800000 0.330372195617

best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 2, 1, 1] 132
cases of this type: 88
best so far: 0
type: [3, 44, 1, 1, 1] 132
cases of this type: 44
best so far: 0
type: [4, 1, 1, 1, 33] 132
cases of this type: 1185921
1266100000 0.025514942237390415
1266200000 0.07641993380307488
1266300000 0.0604501835797131
1266400000 0.041727986992627385
1266500000 0.05000619785817133
1266600000 0.029880650217592716
1266700000 0.008418096493575309
1266800000 0.02157620192605588
1266900000 0.035502555272489786
1267000000 0.04528397996003098
1267100000 0.0367040221